In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

def create_corpus(tweets):
    stop=set(stopwords.words('english'))
    corpus=[]
    for tweet in tqdm(tweets):
#         words=[word.lower() for word in nltk.casual_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        words=[word.lower() for word in nltk.casual_tokenize(tweet)]
        corpus.append(words)
    return corpus

corpus=create_corpus(olid['tweet_cleaned'])

embedding_dict={}
# with open('./data/glove6B/glove.6B.100d.txt','r') as f:
with open('./data/gloveTwitter27B/glove.twitter.27B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus) #??
sequences=tokenizer_obj.texts_to_sequences(corpus) #??

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec
        
        
model=Sequential()

embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=True)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=1e-4)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, to_file='multichannel.png')

In [ ]:
# train=tweet_pad[:tweet.shape[0]]
# test=tweet_pad[tweet.shape[0]:]
le = LabelEncoder()
y = le.fit_transform(olid['subtask_a'])

X_train, X_valid, y_train, y_valid = train_test_split(tweet_pad, y, test_size=0.33, random_state=42)



# X_train,X_test,y_train,y_test=train_test_split(train,tweet['target'].values,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_valid.shape)
callback = EarlyStopping(monitor='val_loss', patience=10, min_delta = 0.005 )

history=model.fit(X_train,y_train,
                  batch_size=512,
                  epochs=200,
                  validation_data=(X_valid,y_valid),
                  shuffle = True,
                  verbose=2,
                 callbacks = [callback])

preds_valid = np.round(model.predict(X_valid))
proba_valid = model.predict_proba(X_valid)
preds_train = np.round(model.predict(X_train))
proba_train = model.predict_proba(X_train)

print(classification_report(y_train, preds_train))
print('AUC: ', roc_auc_score(y_train, proba_train))

print(classification_report(y_valid, preds_valid))
print('AUC: ',  roc_auc_score(y_valid, proba_valid))



In [ ]:
le = LabelEncoder()
y = le.fit_transform(olid['subtask_a'])

y_test = le.transform(olid_test['subtask_a'])
corpus_test = create_corpus(olid_test['tweet_cleaned'])
sequences_test = tokenizer_obj.texts_to_sequences(corpus_test) #??

X_test = pad_sequences(sequences_test,maxlen=MAX_LEN,truncating='post',padding='post')

callback = EarlyStopping(monitor='val_loss', patience=10, min_delta = 0.005 )
history=model.fit(tweet_pad,y,
                  batch_size=512,
                  epochs=100,
                  validation_data=(X_test,y_test),
                  shuffle = True,
                  verbose=2,
                 callbacks = [callback])



print(X_test.shape, y_test.shape)
preds_test = model.predict(X_test)
print(classification_report(y_test, np.round(preds_test)))

print('AUC: ',  roc_auc_score(y_test, preds_test))